In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading in the data from the csv files.

In [ ]:
df_gamedata=pd.read_csv('../input/nfl-big-data-bowl-2022/games.csv')
df_gamedata.head()

In [ ]:
df_plays=pd.read_csv('../input/nfl-big-data-bowl-2022/plays.csv')
df_plays.head()

In [ ]:
df_players=pd.read_csv('../input/nfl-big-data-bowl-2022/players.csv')
df_players.head()

In [ ]:
df_scouting=pd.read_csv('../input/nfl-big-data-bowl-2022/PFFScoutingData.csv')
df_scouting.head()

# Reading in the tracking data for the year 2018.

### In this notebook,in general, I have worked upon data from the year 2018. However, the same could be done for the years 2019 and 2020.

In [ ]:
df_t2018=pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2018.csv')

In [ ]:
df_t2018.head()

In [ ]:
df_players.head()

## Some info regarding the data types and the number of null-values we have in the game data file.

In [ ]:
df_gamedata.info()

## Play data regarding the specific game ID of 2018090600

In [ ]:
df_plays[df_plays['gameId']==2018090600].head()

In [ ]:
df_plays[df_plays['gameId']==2018090600][:1]['playDescription']

## Displaying the 1st 5 rows of the tracking data of the year 2018

In [ ]:
df_t2018.head()

In [ ]:
df_t2018[df_t2018['displayName']=='Justin Tucker']

# Video output for ingame player movement

In [ ]:
from PIL import Image
from skimage import io
f='https://upload.wikimedia.org/wikipedia/commons/thumb/c/c5/AmFBfield.svg/1200px-AmFBfield.svg.png'
a = io.imread(f)
# determining the height ratio
# resize image and save
from PIL import Image
from matplotlib import cm
import matplotlib.pyplot as plt
im = Image.fromarray(a)
im = im.resize((120,53))
plt.imshow(im)
im.save('sompic.png') # Saving the resized background image

In [ ]:
t=df_t2018.loc[(df_t2018['time'].str.contains('2018-12-30',case=False))]

In [ ]:
t.head(10)

# A sample Image for a particular datetime value and a particular match id ingame

In [ ]:
import time
from datetime import datetime, date, time, timedelta
t['time'][len(t)-1]
time_string = "2018-12-30 21:25:32.200"
format_string = "%Y-%m-%d %H:%M:%S.%f"

for i in range(100):
    date_object = datetime.strptime(time_string, format_string)
    date_object = date_object + timedelta(milliseconds=100)
    time_string=str(date_object)
    if '.' in time_string:
        time_string=time_string[:-3]
    else:
        time_string=time_string+'.000'
    final_string=time_string.split(' ')
    final_string=final_string[0]+'T'+final_string[1]
    print(final_string)
    break
print(len(t['time'].unique()))
z=t.loc[(t['time'].str.contains('2018-12-30T21:25:32.200')) & (t['gameId']==2018123000)]
z.index = range(len(z))
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True
f='sompic.png'
a = io.imread(f)
#plt.imshow(a)
fig=plt.figure()
ax = fig.add_subplot(111)
a = io.imread(f)
print(len(z))
for i in range(len(z)):
    ax.imshow(a)
    if z[i:i+1]['team'][i]=='home': 
        ax.plot(z[i:i+1]['x'][i],z[i:i+1]['y'][i],'ro',color='blue')
    elif z[i:i+1]['team'][i]=='away':
        ax.plot(z[i:i+1]['x'][i],z[i:i+1]['y'][i],'ro',color='red')
    else:
        ax.plot(z[i:i+1]['x'][i],z[i:i+1]['y'][i],'ro',color='yellow')
ax.set_xlim(0, 120)
ax.set_ylim(0, 53.3)
ax.axis('on')
leg = ax.legend(loc='upper center')
plt.show()

## Real in-game video analysis

## Here, I have provided video analysis of in-game footage of the players and the movement of the ball throughout the game. The ball is denoted by the yellow dot and the home team is denoted by the blue dots and the away team by red dots.
You can check the entire video out by downloading it after running the code from the /kaggle/working directory. Else, you could just (copy+paste) the following youtube link to view the video: https://youtu.be/boLcsod1toE

In [ ]:
import time
from datetime import datetime, date, time, timedelta

In [ ]:
time_string = "2018-12-30" #given this date 
game_id= 2018123000 #given this gameID

In [ ]:
t=df_t2018.loc[(df_t2018['time'].str.contains('2018-12-30')) & (df_t2018['gameId']==2018123000)]

In [ ]:
times=t['time'].unique()
print(len(times))
print(times)

In [ ]:
from matplotlib.pyplot import plot, draw, show
import cv2
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True
f='sompic.png'
img=[]
ct=0
for i in times:
    fig=plt.figure()
    ax = fig.add_subplot(111)
    a = io.imread(f)
    time_string = i
    time_string=time_string.split('T')
    time_string=time_string[0]+' '+time_string[1]
    format_string = "%Y-%m-%d %H:%M:%S.%f"
    z=t.loc[(t['time'].str.contains(i)) & (t['gameId']==game_id)]
    z.index = range(len(z))
    for i in range(len(z)):
        ax.imshow(a)
        if z[i:i+1]['team'][i]=='home':
            ax.plot(z[i:i+1]['x'][i],z[i:i+1]['y'][i],'ro',color='blue')
        elif z[i:i+1]['team'][i]=='away':
            ax.plot(z[i:i+1]['x'][i],z[i:i+1]['y'][i],'ro',color='red')
        else:
            ax.plot(z[i:i+1]['x'][i],z[i:i+1]['y'][i],'ro',color='yellow')
    ax.set_xlim(0, 120)
    ax.set_ylim(0, 53.3)
    ax.axis('on')
    leg = ax.legend(loc='upper center')
    plt.savefig('Img'+str(i)+'.png')
    img.append(cv2.imread('Img'+str(i)+'.png'))
    os.remove('Img'+str(i)+'.png')
    plt.close(fig)
    ct+=1
height,width,layers=img[0].shape
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video=cv2.VideoWriter('video_match.mp4',fourcc,1,(width,height))
for j in range(0,ct):
    video.write(img[j]) 

In [ ]:
import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLink
FileLink('video_match.mp4')

# Video output for individual player movement during a specific game on a certain day

#### Here, I used a background I found online and resized it in order to help us better understand where the player might've been on the football pitch. At each instant in every graph, there are 2 arrows pointed in the angle of player motion and player orientation respectively. In every graph, the speed and accuracy along with the time is mentioned.

 The arrows essentially represent the tangent to a point on the circle whose length varies. Each plot is saved and read in as an image in an array which is later converted into a video where each frame is a graph at a particular instant of the required match and player on the specified day.

In [ ]:
from PIL import Image
from skimage import io
f='https://upload.wikimedia.org/wikipedia/commons/thumb/c/c5/AmFBfield.svg/1200px-AmFBfield.svg.png'
a = io.imread(f)
# determining the height ratio
# resize image and save
from PIL import Image
from matplotlib import cm
import matplotlib.pyplot as plt
im = Image.fromarray(a)
im = im.resize((120,53))
plt.imshow(im)
im.save('sompic.png') # Saving the resized background image

## Just some sample data I have taken in this case. This could be implemented on any player in any year for any particular scouting agent that is looking for some player data and want to easily analyze the player movement on the pitch as per his/her needs.

In [ ]:
t=df_t2018.loc[(df_t2018['time'].str.contains('2018-12-30',case=False)) & (df_t2018['displayName']=='Justin Tucker')]

In [ ]:
t.index = range(851)

In [ ]:
t[0:1]['time'][0]

## Creating a function to create a graph of the player at every instant in the game and combine it in 1 video for easy analysis

In [ ]:
from matplotlib.pyplot import plot, draw, show
import cv2
img=[]
def picperfect(df):
    plt.rcParams["figure.figsize"] = [7.50, 3.50]
    plt.rcParams["figure.autolayout"] = True
    f='sompic.png'
    a = io.imread(f)
    plt.imshow(a)
    for i in range(len(df)):
        fig=plt.figure()
        ax = fig.add_subplot(111)
        a = io.imread(f)
        ax.imshow(a)
        heading=df[i:i+1]['displayName'][i]+'s='+str(df[i:i+1]['s'][i])+' a='+str(df[i:i+1]['a'][i])+' Time='+ str(df[i:i+1]['time'][i])
        ax.plot(df[i:i+1]['x'][i],df[i:i+1]['y'][i],'ro',label=heading)
        ax.arrow(df[i:i+1]['x'][i],df[i:i+1]['y'][i],2,2*np.tan(np.pi*df[i:i+1]['o'][i]/180),head_width=1, head_length=2, color='blue')
        if df[i:i+1]['playDirection'][i]=='right':
            ax.arrow(10,10,2,2*np.tan(np.pi*0/180),head_width=1, head_length=2, color='yellow', label='Play_Direction='+'right')
        else:
            ax.arrow(10,10,2,2*np.tan(np.pi*0/180),head_width=1, head_length=2, color='yellow', label='Play_Direction='+'left')
        ax.arrow(df[i:i+1]['x'][i],df[i:i+1]['y'][i],2,2*np.tan(np.pi*df[i:i+1]['dir'][i]/180),head_width=1, head_length=2, color='red')
        ax.arrow(df[i:i+1]['x'][i],df[i:i+1]['y'][i],2,2*np.tan(np.pi*df[i:i+1]['o'][i]/180),head_width=0.05, head_length=0.1)
        ax.set_xlim(0, 120)
        ax.set_ylim(0, 53.3)
        ax.axis('on')
        leg = ax.legend(loc='upper center')
        #plt.show()
        plt.savefig('Img'+str(i)+'.png')
        img.append(cv2.imread('Img'+str(i)+'.png'))
        os.remove('Img'+str(i)+'.png')
        plt.close(fig)
    height,width,layers=img[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    video=cv2.VideoWriter('video.mp4',fourcc,1,(width,height))
    for j in range(0,len(df)):
        video.write(img[j])  # Saving all the images in a video of the '.mp4' format

## Using the picperfect() function to create a video of the player's movement

In [ ]:
picperfect(t)

In [ ]:
print(t[105:106]['displayName'][105])

## The video generated could be accessed through the link in the below cell. However, if that doesn't work, you could download it from the /kaggle/working directory after running the code. 
Or,you can just view it right here: https://youtu.be/x86Dtk0n3x0

In [ ]:
#!pip install google.colab
#from google.colab import files
#files.download('../input/nfl-big-data-bowl-2022/games.csv')
import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLink
FileLink('video.mp4')

# Some EDA on some useful/insightful data

## The Height distribution

In [ ]:
df_players.head()

In [ ]:
check = df_players['height'].str.split('-', expand=True)
check.columns = ['first', 'second']
check.loc[(check['second'].notnull()), 'first'] = check[check['second'].notnull()]['first'].astype(np.int16) * 12 + check[check['second'].notnull()]['second'].astype(np.int16)
df_players['height'] = check['first']
df_players['height'] = df_players['height'].astype(np.float32)
df_players['height'] /= 12
df_players.head()

In [ ]:
import seaborn as sns
plt.figure(figsize=(5, 4))

ax = sns.distplot(df_players['height'], bins=10)
ax.set_title('Height Distribution among NFL players');

## Weight distribution

In [ ]:
plt.figure(figsize=(5, 4))

ax = sns.distplot(df_players['weight'], bins=10)
ax.set_title('Weight Distribution among NFL players');

## Player birth year distribution

### This column has some null values. Hence, we shall get rid of that.

In [ ]:
df_players.info()

In [ ]:
df_players.dropna(subset=['birthDate'], inplace=True)

In [ ]:
df_players['birthDate']

In [ ]:
for index, row in df_players.iterrows():
    if len(row['birthDate'].split('/'))>2: 
        df_players.loc[index, 'birthYear'] = row['birthDate'].split('/')[2]
    elif len(row['birthDate'].split('-'))>2: 
        df_players.loc[index, 'birthYear'] = row['birthDate'].split('-')[0]

In [ ]:
plt.figure(figsize=(5, 4))

ax = sns.distplot(df_players['birthYear'], bins=10)
ax.set_title('Birth year Distribution of NFL Players');

## Play Result Distribution

In [ ]:
plt.figure(figsize=(5, 4))
ax = sns.distplot(df_plays['playResult'], bins=10);
ax.set_title('playResult Distribution'); 

## preSnap Home Score Distribution

In [ ]:
plt.figure(figsize=(5, 4))
ax = sns.distplot(df_plays['preSnapHomeScore'], bins=10);
ax.set_title('preSnap_Home_Score Chart in the NFL'); 

## preSnap Visitor Score Distribution

In [ ]:
plt.figure(figsize=(5, 4))
ax = sns.distplot(df_plays['preSnapVisitorScore'], bins=10);
ax.set_title('preSnap_Visitor_Score Graph in the NFL'); 

## Some visualizations about Scouting data

In [ ]:
df_scouting.head()

In [ ]:
df_scouting.info()

### The number of kicktypes and how frequently each one is implemented in the game

In [ ]:
fig, ax = plt.subplots()
df_scouting['kickType'].value_counts().plot(ax=ax, kind='bar')

## I thought of implementing the pairplots here for data as they end up giving us very insightful observations into the dataset as a whole

### A pairplot of the scouting data with the hue as 'kicktype'

In [ ]:
sns.pairplot(df_scouting, hue='kickType')

### Pairplot for the gamedata file with the home team abbreviation as the hue

In [ ]:
sns.pairplot(df_gamedata, hue='homeTeamAbbr')

### A pairplot for the gameplay information file with the hue as the quarter of the game

In [ ]:
sns.pairplot(df_plays,hue='quarter')

# Violin Plots and Box Plots

## Here are some plots for comparing various parameters available in the datasets provided, such as kickLength, kickReturnYardage, hangTime, kickType, yardsToGo, etc.

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")
ax = sns.violinplot(x=df_plays['quarter'])

In [ ]:
sns.set_theme(style="whitegrid")
ax = sns.violinplot(x=df_plays['yardsToGo'],y=df_plays['specialTeamsPlayType'])

In [ ]:
sns.set_theme(style="whitegrid")
ax = sns.violinplot(x=df_plays['kickLength'],y=df_plays['kickReturnYardage'])

In [ ]:
sns.set_theme(style="whitegrid")
ax = sns.violinplot(x=df_scouting['hangTime'],y=df_scouting['kickContactType'])

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")
ax = sns.boxplot(x=df_scouting["hangTime"], y=df_scouting["kickType"])

In [ ]:
sns.set_theme(style="whitegrid")
ax = sns.boxplot(x=df_players["height"], y=df_players["Position"])

## Finally, this is a work in progess and I am working on it to make it better/ get any more meaningful outputs from the data provided. Thanks for viewing my work and please feel free to upvote it and let me know about what more could be worked upon.